# Sækjum gildin fyrir núverandi control methods

In [1]:
import sys
import os

os.chdir("/Users/frodibrooks/Desktop/DTU/Thesis/OptimisedHeating/AgentV2")

In [7]:
from pump_env_demands import WdsWithDemand
import pandas as pd
import numpy as np

def get_state_from_input_named(demand: float, pumpgroup_speeds: list[float]) -> pd.DataFrame:
    assert len(pumpgroup_speeds) == 3, "Expected 3 pump group speeds"

    env = WdsWithDemand(demand_pattern=np.array([demand]), use_constant_demand=True)
    env.reset()

    # Set pump group speeds
    for group_idx, speed in enumerate(pumpgroup_speeds):
        for pump_id in env.pumpGroups[group_idx]:
            env.pump_speeds[pump_id] = speed
            env.wds.pumps[pump_id].speed = speed

    # Apply scaled demand
    for junction in env.wds.junctions:
        junction.basedemand = env.demandDict[junction.uid] * demand

    # Solve and compute state
    env.wds.solve()
    env.pump_power()
    env.calculate_pump_efficiencies()

    # Use get_state_named instead of get_state
    named_state = env.get_state_named()

    # Add demand and pump group speeds to state
    named_state['input_demand'] = demand
    for i, speed in enumerate(pumpgroup_speeds):
        named_state[f'pumpgroup_{i+1}_speed'] = speed

    # Return as DataFrame
    return pd.DataFrame([named_state])


In [20]:
df= get_state_from_input_named(0.898, [1.005, 1.0195, 0.855])
# df= get_state_from_input_named(0.915, [1.01, 0.92, 0.92])

junctions_of_interest = [
    '17_pressure',
    '16_pressure',
    '100_pressure',
    '9_pressure',
    '101_pressure'
]
for junction in junctions_of_interest:
    if junction in df.columns:
        print(f"{junction}: {df[junction].values[0]}")
    else:
        print(f"{junction} not found in DataFrame")

# print(f"Pump Power: {pump_power}")
# 7.8 bar eða 79.5m

# 9 bar eða 91.77

# 5.6 eða 57.1


17_pressure: 79.6847577584036
16_pressure: 79.6825197703856
100_pressure: 91.58697727923779
9_pressure: 91.54910337115085
101_pressure: 57.03908066918662


In [21]:

junctions_of_interest = [
    '17_power',
    '10_power',
    '25_power',
    '26_power',
    '27_power'
]
total_power = 0
for junction in junctions_of_interest:
    if junction in df.columns:
        print(f"{junction}: {df[junction].values[0]}")
        total_power += df[junction].values[0]
    else:
        print(f"{junction} not found in DataFrame")
print(f"Total Power: {total_power}")
# print(f"Pump Power: {pump_power}")
# 7.8 bar eða 79.5m

# 9 bar eða 91.77

# 5.6 eða 57.1


17_power: 22.368416517619867
10_power: 22.39121059934906
25_power: 8.255450368537478
26_power: 8.260005879050636
27_power: 7.89180665426843
Total Power: 69.16689001882548


In [10]:
for column in df.columns:
    print(column)

Junction-11333_pressure
Junction-11333_head
Junction-11333_demand
Junction-9867_pressure
Junction-9867_head
Junction-9867_demand
Junction-41889_pressure
Junction-41889_head
Junction-41889_demand
Junction-58243_pressure
Junction-58243_head
Junction-58243_demand
Junction-470196_pressure
Junction-470196_head
Junction-470196_demand
Junction-33290_pressure
Junction-33290_head
Junction-33290_demand
Junction-22893_pressure
Junction-22893_head
Junction-22893_demand
Junction-477135_pressure
Junction-477135_head
Junction-477135_demand
Junction-33066_pressure
Junction-33066_head
Junction-33066_demand
Junction-32980_pressure
Junction-32980_head
Junction-32980_demand
Junction-54141_pressure
Junction-54141_head
Junction-54141_demand
Junction-42448_pressure
Junction-42448_head
Junction-42448_demand
Junction-476666_pressure
Junction-476666_head
Junction-476666_demand
Junction-42328_pressure
Junction-42328_head
Junction-42328_demand
Junction-48813_pressure
Junction-48813_head
Junction-48813_demand
Junc